In [59]:
import pandas as pd
import os

In [61]:
path='Dataset'
os.listdir(path)

['output.csv', 'output.zip']

In [63]:
for i in os.listdir(path):
    if i is None:
        print("This File Cannot be Found")
    else:
        df=pd.read_csv(os.path.join(path,i))

In [65]:
df.head(n=3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [67]:
df.shape

(39999, 2)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39999 entries, 0 to 39998
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     39999 non-null  object
 1   sentiment  39999 non-null  object
dtypes: object(2)
memory usage: 625.1+ KB


In [71]:
df.describe()

,review,sentiment
count,39999,39999
unique,39733,2
top,Loved today's show!!! It was a variety and not...,negative
freq,5,20006


In [73]:
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [75]:
boolean = df.duplicated().any() 
boolean

False

In [77]:
df.drop_duplicates(keep=False, inplace=True)

In [79]:
df.describe()

,review,sentiment
count,39476,39476
unique,39476,2
top,One of the other reviewers has mentioned that ...,positive
freq,1,19851


In [81]:
print(len(df['review'].unique()),df['sentiment'].count())

39476 39476


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Apply preprocessing to the dataset
df['preprocessed_review'] = df['review'].apply(preprocess_text)

In [ ]:
a=df['preprocessed_review'].tolist()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Feature extraction
def extract_features(corpus):
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer to the corpus and transform the corpus
    features = vectorizer.fit_transform(corpus)

    return features

corpus = df['preprocessed_review'].tolist()
features = extract_features(corpus)
print("Shape of features:", features.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into train and test sets
X = features
y = df['sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

# Train a logistic regression model
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm=confusion_matrix(y_test,y_pred)
disp=ConfusionMatrixDisplay(cm,display_labels=["Positive","Negative"])
disp.plot()
plt.show()

In [ ]:
#this is a test file 'test.csv' from kaggle 

In [ ]:
my_test_file = pd.read_csv('test.csv')
my_test_file 

In [ ]:
def preprocess_text2(text):
    if isinstance(text, float):  # Check if the value is float (NaN)
        return ''
    
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Apply preprocessing to the test dataset
my_test_file['preprocessed_text'] = my_test_file['text'].apply(preprocess_text2)
my_test_file['preprocessed_text'] 

In [ ]:
# Extract the preprocessed reviews from the test dataset
my_test_corpus = my_test_file['preprocessed_text'].tolist()

In [ ]:
# Create a new vectorizer object and fit it with the training data vocabulary
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

In [ ]:
# Transform the test data using the fitted vectorizer
my_test_features = vectorizer.transform(my_test_corpus)

In [ ]:
# Predict the labels for the test data
my_test_predictions = model.predict(my_test_features)
my_test_predictions

In [ ]:
my_test_file

In [ ]:
# Save the predictions to a file ALso the file didn't have the sentiment labels associated 
output_file = 'test_predictions.csv'
my_test_file['predicted_sentiment'] = my_test_predictions
my_test_file.to_csv(output_file, index=False)
print("Predictions saved to:", output_file)